# Benchmarking OPE on Amazon Music Data

This notebook performs _off-policy evaluation_ (OPE) using only the logged data. The policy (model) artifacts that generated the data are not needed for evaluation, as we have logged the selected actions from both the logging policy and target policy, for every context in the dataset. We also store the full propensity table (probability of ranking each item at each position) under the logging policy. The target policy is deterministic, so its propensities are not needed.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

np.random.seed(42)

In [ ]:
import os

# D1_DATASET_PATH = "/Users/vitob/DataspellProjects/OPE/dataset"
# D2_DATASET_PATH = "/Users/vitob/DataspellProjects/OPE/dataset"

D1_DATASET_PATH = f"{os.getcwd()}/D1"
D2_DATASET_PATH = f"{os.getcwd()}/D2"

# Position Bias curve
PB = np.array(
    [
        1.0,
        0.76674848,
        0.33742719,
        0.26136553,
        0.20732642,
        0.16384161,
        0.13263467,
        0.11458964,
        0.11255035,
        0.09426026,
        0.07283572,
        0.07004059,
        0.06315475,
        0.05890552,
        0.05496263,
    ]
)

### Ground Truth

First, we compute the "ground truth" for D1 using the average of its logged rewards.

In [ ]:
%%time

from estimators.ground_truth import GroundTruth

result_ground_truth = GroundTruth(omega=1.018306600302338).evaluate(D1_DATASET_PATH)

In [ ]:
print(result_ground_truth)

### Estimates

For each estimator, compute a reward estimate and confidence interval.

In [ ]:
%%time

from joblib import Parallel, delayed

from estimators.ipm import IPM
from estimators.pbm import PBM, PBMType
from estimators.interpol import INTERPOL

estimators = [
    ("PBM-D", PBM(position_bias=PB, type=PBMType.DETERMINISTIC)),
    ("PBM-S", PBM(position_bias=PB, type=PBMType.STOCHASTIC)),
    ("IPM", IPM()),
    ("Interpol(window=1)", INTERPOL(window_size=1, position_bias=PB)),
    ("Interpol(window=2)", INTERPOL(window_size=2, position_bias=PB)),
    ("Interpol(window=3)", INTERPOL(window_size=3, position_bias=PB)),
    ("Interpol(window=4)", INTERPOL(window_size=4, position_bias=PB)),
    ("Interpol(window=5)", INTERPOL(window_size=5, position_bias=PB)),
]

all_results = Parallel(n_jobs=10)(
    delayed(estimator.benchmark)(name, D2_DATASET_PATH, result_ground_truth)
    for (name, estimator) in estimators
)

rows = []
for name, results, err in all_results:
    rows.append(
        {
            "estimator": name,
            "metric": results.metric,
            "err_lower": results.metric - results.ci.lower_bound,
            "err_upper": results.ci.upper_bound - results.metric,
            "lower_bound": results.ci.lower_bound,
            "upper_bound": results.ci.upper_bound,
            "ground_truth": result_ground_truth.metric,
            "bias2": err.bias2,
            "var": err.var,
            "mse": err.mse,
        }
    )

### Visualize Results

In [ ]:
df = pd.DataFrame(rows).set_index("estimator")
df.to_csv("OPE.csv", index_label="estimator")

In [ ]:
df

In [ ]:
ax = df.plot.bar(
    y="metric",
    yerr=[df["err_lower"], df["err_upper"]],
    capsize=3,
    rot=90,
    ylabel=r"$\hat{V}(\pi)$",
    legend=False,
).axhline(y=df.iloc[0]["ground_truth"], color="r", linestyle="-")

ax.figure.get_axes()[0].legend([r"$\hat{V} (\pi)$", r"$V(\pi)$"])

ax.figure.tight_layout()
ax.figure.savefig("OPE.png", bbox_inches="tight")

In [ ]:
df_err = df[["bias2", "var", "mse"]]
ax = df_err.plot.bar()

ax.figure.get_axes()[0].legend([r"Bias$^2$", r"Var", r"MSE"])

ax.figure.tight_layout()
ax.figure.savefig("OPE-error_decomposition.png", bbox_inches="tight")

### Sample size

In [ ]:
rows = []

for sample_size in range(100_000, 900_000, 100_000):
    sample_size_bootstrap_results = Parallel(n_jobs=10)(
        delayed(estimator.benchmark)(
            name, D2_DATASET_PATH, result_ground_truth, sample_size
        )
        for (name, estimator) in estimators
    )

    for name, results, err in sample_size_bootstrap_results:
        rows.append(
            {
                "estimator": name,
                "sample_size": sample_size,
                "metric": results.metric,
                "lower_bound": results.ci.lower_bound,
                "upper_bound": results.ci.upper_bound,
                "err_lower": results.metric - results.ci.lower_bound,
                "err_upper": results.ci.upper_bound - results.metric,
                "ground_truth": result_ground_truth.metric,
                "bias2": err.bias2,
                "var": err.var,
                "mse": err.mse,
                "mse_lower_bound": err.ci.lower_bound,
                "mse_upper_bound": err.ci.upper_bound,
            }
        )

In [ ]:
df = pd.DataFrame(rows)
df.to_csv("OPE-SampleSize.csv", index=False)

In [ ]:
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

sample_sizes = sorted(df["sample_size"].unique())
estimators = df["estimator"].unique()
n_estimators = len(estimators)

# Create positions for each sample size block
x_positions = np.arange(len(sample_sizes))
width = 0.8 / n_estimators  # width of each estimator bar within a block

for i, est in enumerate(estimators):
    subdf = df[df["estimator"] == est]

    # Position within each sample size block
    x_offset = (i - n_estimators / 2 + 0.5) * width
    x_pos = []
    y_vals = []
    y_errs = []

    for size in sample_sizes:
        size_data = subdf[subdf["sample_size"] == size]
        if not size_data.empty:
            x_pos.append(x_positions[sample_sizes.index(size)] + x_offset)
            y_vals.append(size_data["metric"].iloc[0])
            y_errs.append(
                [
                    size_data["metric"].iloc[0] - size_data["lower_bound"].iloc[0],
                    size_data["upper_bound"].iloc[0] - size_data["metric"].iloc[0],
                ]
            )

    if y_errs:
        ax.errorbar(
            x_pos, y_vals, yerr=np.array(y_errs).T, fmt="o", capsize=4, label=est
        )

# Horizontal line for ground truth
ax.axhline(df["ground_truth"].iloc[0], color="red", linestyle="--", label=r"$V(\pi)$")

# Formatting
ax.set_xticks(x_positions)
ax.set_xticklabels(sample_sizes)
ax.set_xlabel("Dataset size")
ax.set_ylabel(r"$\hat{V}(\pi)$")

ax.legend(loc="lower right", ncol=3)

ax.figure.tight_layout()
ax.figure.savefig("OPE-SampleSize.png", bbox_inches="tight")